# FastAPI Inference Service - Colab Demo

This notebook demonstrates how to use the FastAPI inference service in Google Colab.

## Prerequisites

1. Clone the repository to Colab
2. Have trained Stage1 and Residual Boost models
3. Have evaluation and inference data

## Step 1: Setup Environment

In [ ]:
# Install dependencies (if not already installed)
!pip install fastapi uvicorn pydantic httpx -q

In [ ]:
# Change to repo directory
%cd /content/Industrial-digital-twin-by-transformer

## Step 2: Start FastAPI Service

In [ ]:
# Start FastAPI service in background
!nohup python -m fastapi_inference.main > fastapi.log 2>&1 &

# Wait for service to start
import time
print("Waiting for service to start...")
time.sleep(5)
print("Service should be ready!")

In [ ]:
# Check service logs
!tail -20 fastapi.log

## Step 3: Test Service Health

In [ ]:
import requests
import json

BASE_URL = "http://localhost:8000"

# Health check
response = requests.get(f"{BASE_URL}/api/v1/health")
print(json.dumps(response.json(), indent=2))

## Step 4: Load Models

**⚠️ Important: Update the paths below with your actual model paths!**

In [ ]:
# UPDATE THESE PATHS WITH YOUR ACTUAL MODEL PATHS
stage1_config = "saved_models/your_sst_model_inference.json"
residual_boost_config = "saved_models/tft_models/your_tft_inference.json"

# Load Stage1 model
response = requests.post(
    f"{BASE_URL}/api/v1/models/stage1/load",
    json={"inference_config_path": stage1_config}
)

if response.status_code == 200:
    stage1_info = response.json()
    print(f"✅ Stage1 loaded: {stage1_info['model_name']}")
    print(f"   Boundary signals: {stage1_info['num_boundary_signals']}")
    print(f"   Target signals: {stage1_info['num_target_signals']}")
else:
    print(f"❌ Failed to load Stage1: {response.text}")

In [ ]:
# Load Residual Boost model
response = requests.post(
    f"{BASE_URL}/api/v1/models/residual-boost/load",
    json={"inference_config_path": residual_boost_config}
)

if response.status_code == 200:
    rb_info = response.json()
    print(f"✅ Residual Boost loaded: {rb_info['model_name']}")
    print(f"   Boundary signals: {rb_info['num_boundary_signals']}")
    print(f"   Target signals: {rb_info['num_target_signals']}")
else:
    print(f"❌ Failed to load Residual Boost: {response.text}")

In [ ]:
# List all loaded models
response = requests.get(f"{BASE_URL}/api/v1/models/list")
models = response.json()
print(json.dumps(models, indent=2))

## Step 5: Create Ensemble

**⚠️ Update the evaluation data path below!**

In [ ]:
# UPDATE THIS PATH
evaluation_data = "data/your_evaluation_data.csv"

# Create ensemble
response = requests.post(
    f"{BASE_URL}/api/v1/ensemble/create",
    json={
        "stage1_model_name": stage1_info['model_name'],
        "residual_boost_model_name": rb_info['model_name'],
        "evaluation_data_path": evaluation_data,
        "delta_r2_threshold": 0.05,
        "save_config": True
    }
)

if response.status_code == 200:
    ensemble_info = response.json()
    ensemble_name = ensemble_info['ensemble_name']
    print(f"✅ Ensemble created: {ensemble_name}")
    print(f"   Delta R² threshold: {ensemble_info['delta_r2_threshold']}")
    print(f"   Signals using Residual Boost: {ensemble_info['num_use_boost']}")
    print(f"   Signals using Stage1 only: {ensemble_info['num_use_stage1_only']}")
    print(f"\nPerformance Metrics:")
    print(f"   Stage1 R²: {ensemble_info['metrics']['stage1']['r2']:.4f}")
    print(f"   Ensemble R²: {ensemble_info['metrics']['ensemble']['r2']:.4f}")
    print(f"   Improvement: {ensemble_info['metrics']['improvement']['r2_pct']:.2f}%")
else:
    print(f"❌ Failed to create ensemble: {response.text}")

In [ ]:
# View signal analysis
if 'ensemble_info' in locals():
    import pandas as pd
    
    signal_df = pd.DataFrame(ensemble_info['signal_analysis'])
    print("\nSignal Analysis:")
    print(signal_df.to_string(index=False))

## Step 6: Batch Inference

**⚠️ Update the inference data path below!**

In [ ]:
# UPDATE THIS PATH
inference_data = "data/your_inference_data.csv"

# Run batch inference
response = requests.post(
    f"{BASE_URL}/api/v1/inference/batch",
    json={
        "ensemble_name": ensemble_name,
        "input_data_path": inference_data,
        "output_dir": "fastapi_inference/results",
        "include_metadata": True
    }
)

if response.status_code == 200:
    result = response.json()
    print(f"✅ Inference completed!")
    print(f"   Output: {result['output_path']}")
    print(f"   Samples processed: {result['num_samples']}")
    print(f"   Signals: {result['num_signals']}")
    print(f"   Signals used Residual Boost: {result['num_signals_used_boost']}")
else:
    print(f"❌ Inference failed: {response.text}")

In [ ]:
# View predictions
if 'result' in locals():
    import pandas as pd
    
    pred_df = pd.read_csv(result['output_path'])
    print(f"\nPredictions shape: {pred_df.shape}")
    print(f"\nFirst few rows:")
    print(pred_df.head())

## Step 7: Update Threshold (Optional)

In [ ]:
# Update Delta R² threshold
response = requests.post(
    f"{BASE_URL}/api/v1/ensemble/{ensemble_name}/update-threshold",
    json={"new_threshold": 0.08}
)

if response.status_code == 200:
    updated_info = response.json()
    print(f"✅ Threshold updated to {updated_info['delta_r2_threshold']}")
    print(f"   Signals using Residual Boost: {updated_info['num_use_boost']}")
    print(f"   Signals using Stage1 only: {updated_info['num_use_stage1_only']}")
else:
    print(f"❌ Update failed: {response.text}")

## Step 8: Manual Signal Control (Optional)

In [ ]:
# Run inference with manual signal control
response = requests.post(
    f"{BASE_URL}/api/v1/inference/batch",
    json={
        "ensemble_name": ensemble_name,
        "input_data_path": inference_data,
        "output_dir": "fastapi_inference/results",
        "manual_boost_signals": {
            # UPDATE WITH YOUR SIGNAL NAMES
            "Temperature_1": True,
            "Pressure_2": False
        },
        "include_metadata": True
    }
)

if response.status_code == 200:
    result = response.json()
    print(f"✅ Inference with manual control completed!")
    print(f"   Signals used Residual Boost: {result['signals_used_boost']}")
else:
    print(f"❌ Inference failed: {response.text}")

## Step 9: Cleanup (Optional)

In [ ]:
# Stop FastAPI service
!pkill -f "fastapi_inference.main"
print("Service stopped")